In [3]:
from PIL import Image
from numpy import random
import numpy as np
import scipy.misc
import matplotlib.pyplot as plt
import imageio
import colorsys

In [ ]:
def colorsort(colors):
    hues = [colorsys.rgb_to_hls(*[x[y] / 255.0 for y in range(3)])[0] for x in colors]
    return [colors[x] for x in np.argsort(hues)]

In [4]:
def tricolor(colors, width = 640, height = 480):
    # Image size
    channels = 3

    # Create an empty image
    img = np.zeros((height, width, channels), dtype=np.uint8)

#     colors = []
#     for i in range(3):
#         colors.append([random.randint(0,256) for x in range(3)])

    orientation = random.randint(0,2)
        
    for x in range(width):
        for y in range(height):
            if orientation == 1:
                img[y][x] = colors[int(3 * x / width)]
            else:
                img[y][x] = colors[int(3 * y / height)]

    imageio.imwrite('outfile.png', img)

In [117]:
def palette(colors, width = 64, height = 64):
    # Image size
    channels = 3
    colors = colorsort(colors)
    
    # Create an empty image
    img = np.zeros((height, width * len(colors), channels), dtype=np.uint8)

    for x in range(width * len(colors)):
        for y in range(height):
            img[y][x] = colors[int(x / width)]
            
    imageio.imwrite('outfile.png', img)

In [5]:
def rothko(colors, width = 480, height = 480):
    channels = 3


    # base
    img = np.zeros((height, width, channels), dtype=np.uint8)
    base_col = [int((255 * x)) for x in list(colorsys.hsv_to_rgb(random.uniform(0,1), random.uniform(0,0.8), random.uniform(0,0.5)))]
    for x in range(width):
        for y in range(height):
            img[y][x] = base_col
            
    # number of colors (1-4)
#     num_col = random.randint(1,4)
    num_col = len(colors)
    
    # width padding
    width_pad = int(width * random.uniform(0.05, 0.1))
    
    # height padding
    buckets = [0] * (num_col * 2 + 1)
    for i in range(num_col * 2 + 1):
        if (i % 2) == 0:
            buckets[i] = random.uniform(0.2, 0.1)
        else: 
            buckets[i] = random.lognormal(0.8)
    buckets = np.cumsum(buckets)
    buckets = buckets * height / max(buckets)
    buckets = buckets.astype(int)
    
    reorder = random.randint(0,10)
    
    for x in range(width_pad, width - width_pad):
        for y in range(height):
            col_select = np.argmax(buckets > y) + 1
            if (col_select % 2) != 1:
                img[y][x] = colors[ ((col_select / 2) + reorder) % num_col]
    imageio.imwrite('outfile.png', img)

In [6]:
def triad():
    offset = random.uniform(0.1, 0.4)
    
    hue = np.zeros(3)
    hue[1] = random.uniform(-1,1)
    hue[0] = hue[1] + offset
    hue[2] = hue[1] - offset
    
    colors = []
    for i in range(3):
        if i == 1:
            saturation = random.uniform(0.8,0.1)
            value = random.uniform(0.8,1)
        else: 
            saturation = random.uniform(0.1,0.8)
            value = random.uniform(0.2, 0.8)

        colors.append([int((255 * x)) for x in list(colorsys.hsv_to_rgb(hue[i], saturation, value))])

    return(colors)
    

In [155]:
rothko(triad())

In [130]:
random.lognormal(0.8)

0.3774996185732141

In [9]:
import requests
from io import BytesIO
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
import numpy as np
from PIL import Image

In [64]:
response = requests.get('https://i.redd.it/6sfs5vgo1jd21.jpg', verify=False)
img = Image.open(BytesIO(response.content))
pixels = list(img.getdata())
sample_indices = list(random.choice(len(pixels), size = 10000, replace=False))
sample_pixels = [pixels[i] for i in sample_indices]

/Users/RFBZ/miniconda3/envs/colors/lib/python2.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [68]:
kmeans = KMeans(n_clusters=7, random_state=0).fit(sample_pixels)
kmeans_palette = [[int(x) for x in kmeans.cluster_centers_[y]] for y in range(7)]
palette(kmeans_palette)

In [60]:

gmm = GaussianMixture(n_components = 5).fit(sample_pixels)
centers = []
for i in range(gmm.n_components):
    density = scipy.stats.multivariate_normal(cov=gmm.covariances_[i], mean=gmm.means_[i]).logpdf(gmm_sample)
    centers.append(gmm_sample[np.argmax(density)])
tricolor([centers[i] for i in range(3)])
palette(centers)

In [81]:
[colorsys.rgb_to_hsv(*x) for x in kmeans_palette]

[(0.16666666666666666, 0, 28),
 (0.16666666666666666, 0, 223),
 (0.5, 0, 153),
 (0.16666666666666666, 0, 109),
 (0.5, 0, 128),
 (0.5, 0, 74),
 (0.16666666666666666, 0, 68)]

In [86]:
colorsys.rgb_to_hsv(*kmeans_palette[0])

(0.16666666666666666, 0, 28)

In [106]:
hues = [colorsys.rgb_to_hls(*[x[y] / 255.0 for y in range(3)])[0] for x in kmeans_palette]


IndexError: list index out of range

In [109]:
centers

[(34, 72, 13), (129, 138, 153), (12, 65, 99), (16, 53, 0), (254, 220, 183)]

In [118]:
palette(kmeans_palette)